# PyTorch Basics

Tutorial by Abdulah Fawaz and Emma R

## PyTorch Tensors
<a id='tensors'></a>

PyTorch is a lot like numpy. A lot of operations used to manipulate numpy arrays have their counterparts in pytorch and numpy arrays can be converted to and from pytorch *tensors*. PyTorch arrays are given the more proper mathematical name of tensors (see e.g tensorflow). In terms of practical usage pytorch tensors can be manipulated very similarly to NumPy’s ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing.

Below are some examples. Let us begin by importing torch and numpy.

In [1]:
import torch #import torch
import numpy as np

ModuleNotFoundError: No module named 'torch'

Generating a random array of size 2x2x2: NumPy vs PyTorch

In [ ]:
# Numpy
numpy_random_arr = np.random.rand(2,2,2)
numpy_random_arr

In [ ]:
# PyTorch
torch_random_arr = torch.rand(2,2,2)
torch_random_arr

They are indexed in the same way

In [ ]:
torch_random_arr[0,0,0], numpy_random_arr[0,0,0]

and can be reshaped using reshape functions ...

In [ ]:
torch_random_arr = torch_random_arr.reshape(4,2)
print(torch_random_arr.size())

In [ ]:
numpy_random_arr = np.reshape(numpy_random_arr, [4,2])
print(numpy_random_arr.shape)

Converting to and from numpy arrays is easy:

In [ ]:
a = np.array([[1,2],[3,4]]) # make a numpy array

a_torch = torch.from_numpy(a) #converting to a torch Tensor from a numpy array
print(a_torch) 

a_np = a_torch.numpy() # converting to a numpy array from a torch Tensor
print(a_np)

Other basic functions such as torch.diag, torch.cat (concatenate), torch.matmul work similarly to their numpy equivalents. <br>
As always, when looking for a function **check the [documentation](https://stackoverflow.com/questions/25692293/inserting-a-link-to-a-webpage-in-an-ipython-notebook)** and consider running through the [official pytorch tutorial](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html) on tensor manipulation 

Notice that printing the torch tensor also gave a dtype. Just like in numpy, the data type of an object is important.
PyTorch Tensors types - just like in any other programming language - depend on whether they are storing integers, floating points or bools, and in how many bits. Often, it is important to make sure tensors are of the right / matching type when performing operations on them.
<br>
See https://pytorch.org/docs/stable/tensors.html for a list of dtypes and what they are called in PyTorch.

Changing torch tensor type is simple too:


In [ ]:
print(a_torch)
print(a_torch.to(torch.double)) #casts the int32 dtype tensor into a 64 bit float dtype tensor

Tensor reshaping/resizing can be completed using torch.view:

In [ ]:
x = torch.randn(3, 3)

y = x.view(1, -1)  # the size -1 is inferred from other dimensions
print(x.size(), y.size())


## Exercise 1. Basic Tensor Operations

There is some streamlining of operations in pytorch relative to numpy which can simplify code development. For example, compare here the multiplications of two arrrays using broadcasting.

1. Generate two random numpy arrays, **a** and __b__ of sizes [12,5] and [3,5,20] 


2. Find the matrix product **a** $\cdot$ __b__ using broadcasting. The result should be of shape (3,12,20) <br>
    *hint: may need to reshape a first*
    
    
3. Convert **a** and __b__ into PyTorch Tensors and perform direct multiplication (without reshaping).



In [ ]:
# Solution:
a = np.random.rand(12,5)
b = np.random.rand(3,5,20)
np.matmul(np.reshape(a, [1, 12,5]),b).shape

a_t = torch.from_numpy(a)
b_t = torch.from_numpy(b)

print(torch.matmul(a_t, b_t).size())

## Autograd: automatic differentiation

As you might imagine, it is not the similarities between the two that we are interested in, but what makes torch Tensors relevant to machine learning. The most significant and relevant difference is that PyTorch Tensors also have an associated *gradient*. It is this that is used to perform the optimization that machine learning is based on. 

The gradient of a pytorch tensor is stored as its ```.grad``` attribute. All pytorch tensors have this even if it is not apparent nor used. In such a case it would be set to "None". 

All PyTorch tensors have another boolean attribute ```requires_grad``` that indicates whether pytorch *needs* to track and store its gradient or whether it is simply a static tensor. By default, requires_grad is set to False.
When we later construct neural networks from the torch.nn neural network module, requires_grad will be automatically set to True for the relevant learning parameters so it is not something you should generally worry about setting manually.

In addition the attribute ```grad_fn```: This is the backward function used to calculate the gradient.

For example, let's observe gradient estimation for a simple function $L = \frac{1}{N} \sum_{i}\sum_j (2x_{ij}+3)^2  $ operating on a matrix $\mathbf{X}$ with $N$ elements.

In [ ]:
X = torch.ones(5,5, requires_grad=True) # generate a random Tensor

print(X.grad) # check its gradient - the result is None

y=2*X+3
z=y*y
out = z.mean()


out.backward()
print(X.grad) 
print(out.grad_fn)


### Exercise2  prove this is correct using the chain rule.

Do this now and then try repeating the process for another simple function, of your choice. Try also generating $\mathbf{X}$ in different ways, suign torch random number generators for example.

**Note** PyTorch only supports gradient estimation for can only be calculated for floating point tensors.

In pratice the ```autograd``` package provides an engine to perform backpropagation. As variables and operations are defined it sets up a dynamic computational graph in the same sense as we saw in our first lecture. In this, the leaves of the graph are input tensors, defined using initialisation operations such as those shown [above](#tensors), and identified using the attribute ```is_leaf==True```. Roots are output tensors. Gradients are then calculated by tracing the graph from the root to the leaf and multiplying every gradient in the way using the chain rule. 



## PyTorch NN Functions

In [ ]:
import torch.nn as nn

The torch.nn module contains all the functions you will need to build a neural network. 
<br> This includes fully connected layers, convolutions, and pooling operations. 


It is well documented and easy to read: **See for Yourself** https://pytorch.org/docs/stable/nn.html

We will go over a few nn modules and then use what we have learned to build an MLP.

Let us begin with a 2d Convolution: one of the most common and important functions used in image processing and deep learning in general. We will be generating a *single* 3D 'image' example (array of 3 input channels, 100 x 100 pixels) and performing a 2d Convolution with stride = 1, kernel size = 3x3 and 2 output channels

In [ ]:
# the first dimension has size N where N is the number of images. 
#here it is simply 1

input_image = torch.randint(0, 255, (1, 3,100,100)) # our random image. 

# building our conv operation. note that we did not need to specify the names of the parameters. 
#nn.Conv2d(3,2,3) is sufficient
operation = nn.Conv2d(in_channels = 3,out_channels = 2, kernel_size = 3) 

print(operation) #we can see our convolution operation by printing it

First try the following operation - observe the ```RuntimeError```

In [ ]:
result = operation(input_image)

The operation fails as it cannot work on integer tensors. Let us convert it into a float tensor first


In [ ]:
input_image = input_image.to(torch.float)
result = operation(input_image)
print(result.shape)

Observe the shape of the ```result``` with respect to the shape of the original image. We see we lose a unit around the edge of the 2D image and the output number of features reduce from 3 to 2 as required.

In [ ]:
print(result.shape,input_image.shape)

We can correct this using padding, as:

In [2]:
operation = nn.Conv2d(in_channels = 3,out_channels = 2, kernel_size = 3, padding=1)
input_image = input_image.to(torch.float)
result = operation(input_image)
print(result.shape)

NameError: name 'nn' is not defined

The result shows the result of a 2d Convolution between our image and some randomly generated kernel. What if we wanted to inspect that kernel? We can use: 

In [ ]:
for name, param in operation.named_parameters(): # for each named parameter
    print(name, param.data.shape)

Above we can see that our convolution weight tensor is of shape [2,3,3,3] (2 $3\times 3\times 3$ convoltuional filters) and has a bias of shape [2].

We can do the same for a fully connected Linear layer, which can be found in the torch.nn module under the function Linear(). Its parameters are the number of input features and the number of output features.
We will use 3x100x100 = 300000 input features and 10 output features.

In [ ]:
fc_operation = nn.Linear(30000, 10) # defining our fully connected Linear layer

reshaped_input_image = input_image.reshape(input_image.size(0), -1) #reshaping input image 

print('input image shape ', input_image.shape,'reshaped_input_image shape' , reshaped_input_image.shape)
result = fc_operation(reshaped_input_image) 


The goal with the reshape operation is to unravel the image into a single vector. As you may be accustomed with in numpy, reshape dimensions of (input_image.size(0), -1) reshape the image into a vector, where -1 tells pytorch to, essentially, figure out itself what should be the corresponding size of this dimension given the input. 

Note, input_image.size(0) returns the size of the first dimension of our image array, which represents the number of examples. Here, it is one but in real examples, there can be a variable number of images used in each batch. By defining the first dimension of the reshape size(0) it ensures that we unravel each image independently, to return an $N \times 300000$ array: ```reshaped_input_image```

### nn.Functional

As you will see as we go forward most PyTorch layers can be implemented either as a `torch.nn.Module` object or as a `torch.nn.Functional` function. So which should you use? 

Essentially,  `nn.functional` provides building block functions (e.g. layers / activations) in form of functions. This means that they can be directly called on the input rather than defining the object. 

In cases, where we have weights or other states which might define the behaviour of the layer (for example, a dropout / Batch Norm layer behaves differently during training and inference) a convolutional layer where we need to keep track of the weights over time, then we should use `nn.Module` objects. The whle points is that these define a class to hold the data structure, and make (e.g. convolutional) operations member functions.

On the other hand, in cases where no state or weights are required, `nn.functional` counterparts may be used. Examples being, resizing (nn.functional.interpolate),  average pooling (nn.functional.AvgPool2d) and activation functions.

For more details see https://blog.paperspace.com/pytorch-101-advanced/

## Max pooling

The maxpool function in pytorch is [```nn.MaxPool2d```](https://pytorch.org/docs/stable/nn.html?highlight=maxpool#torch.nn.MaxPool2d). As we have seen in our lectures the max pool operation downsamples an image by selecting the maximum intensity of an image patch to represent the whole patch.

### Exercise 3: MaxPooling in 2D.

Generate a random integer array to represent 5 images which have 3 channels are of size (100 x 100). Perform a 2D maxpool on the images using PyTorch. Your max pooling operation should have:

1. filter size 3x3, stride = 1 x 1

2. filter size 4 x 2, stride = 2 x 2

**Hint** check the docs (linked above)


In [ ]:
#solution

random_ims = torch.randint(0, 255, (5,3,100,100)).to(torch.float)

maxpoolop = nn.MaxPool2d(3,1) 
print(maxpoolop)
r = maxpoolop(random_ims)
print(r.shape)


maxpoolop = nn.MaxPool2d((4,2),2) 
print(maxpoolop)
r = maxpoolop(random_ims)
print(r.shape)

## Data Loading

In Deep Learning, the data must be collected and prepared into batches before being fed into the neural network for training. In many cases, our medical imaging data is too large to be all loaded into memory at once. We may also want to transform (augment) our data either as a pre-processing step or to simulate the creation of bigger data sets. Typically we want to randomly shuffle our data during training such that our network does not always see data in the same order. Pytorch streamlines this process through the provision of two classes: *DataSet*, and accompanying iterator *DataLoader*.

For common datasets such as MNIST and CIFAR10 Pytorch provides default `DataSets` https://pytorch.org/docs/stable/torchvision/datasets.html. However, for more bespoke applications it is necessary to create tailored `DataSet` classes which inherit from the base class.

In [ ]:
from torch.utils.data import Dataset, DataLoader

### The Dataset and DataLoader Class

Let's look at the basic structure of the `Dataset` class (https://github.com/pytorch/pytorch/blob/master/torch/utils/data/dataset.py) and discuss some of it's optional features. 

`class Dataset(object):  
    """An abstract class representing a :class:Dataset.
    All datasets that represent a map from keys to data samples should subclass
    it. All subclasses should overwrite :meth:__getitem__, supporting fetching a
    data sample for a given key. Subclasses could also optionally overwrite
    :meth:__len__/, which is expected to return the size of the dataset by many
    :class:~torch.utils.data.Sampler implementations and the default options
    of :class:~torch.utils.data.DataLoader.
    .. note::
      :class:~torch.utils.data.DataLoader by default constructs a index
      sampler that yields integral indices.  To make it work with a map-style
      dataset with non-integral indices/keys, a custom sampler must be provided.
    """ 
    def __getitem__(self, index):
        raise NotImplementedError
    def __add__(self, other):
        return ConcatDataset([self, other])`

What this states is that any class that inherits from the baseclass must override the following methods:

- `__len__` so that len(dataset) returns the size of the dataset.
- `__getitem__` which returns a sample from the dataset given an index. For supervised learning from images this requires it to return both an example image and its label.

In addition to this it is common to pass a transform argument to the `DataSet` class which will support augmentation of the data. After that you have great freedom as to the actual structure and ordering of the code in the class. 

The `DataLoader` is an iterator class, which uses the `__getitem__` and `__len__` functions to collate data into batches and sample at random (`shuffle`) from the data referenced by the `Dataset` class. It also supports loading and processing the data in parallel (with the number of parallel processes determined by parameter `num_workers`. **Generally shuffling the order of the data is very important** as, in this way, the batches between epochs will not look alike, improving generalisation.

The generic form of a call to `DataLoader` is 

`dataloader = DataLoader(transformed_dataset, batch_size=4,
                        shuffle=True, num_workers=4)`
                        
Unlike the `Dataset` class this is unlikely to need overloading.


### Loading MNIST from torchvision

Let's start by looking at how to use custom PyTorch datasets available through `torchvision.`

The torchvision package consists of popular datasets, model architectures, and common image transformations for performing computer vision tasks.

Let us use it to load the MNIST data set. 

In [ ]:
from torchvision import datasets

MNIST is found under datasets.MNIST. The class supports direct download of the data from the internet as indicated by the `download` argument. It is necessary to define a target directory for the download as `root`. Data is also already separated into `train` and `test` subsets. For further help understanding the class datasets.MNIST, use $\text{datasets.MNIST.__doc __}$ to view the documentation or view https://pytorch.org/docs/stable/torchvision/datasets.html#mnist.

In [ ]:
print(datasets.MNIST.__doc__)

There are many potential transformations that are supported through `torchvision` and can be performed to the image (such as rotations, crops, flips, etc). The use of these is controlled by the `transform` argument of the `Dataset` class. Here, however, we will only apply a transformation to turn the PIL format image into a torch tensor

In [ ]:
from torchvision import datasets
from torchvision import transforms
mnist_train_dataset = datasets.MNIST(root = 'mnist_data/train', download= True, train = True,
                                     transform = transforms.ToTensor())
mnist_test_dataset = datasets.MNIST(root = 'mnist_data/test', download= True, train = False, 
                                    transform = transforms.ToTensor())

To explore the shape of the data set we must used the overloaded class function ```len()``` to determine the number of examples. 

In [ ]:
print(len(mnist_train_dataset))

In this case, as MNIST is an image classfication data set. Each individual item is a tuple, representing the data and its integer label i.e.

In [ ]:
print(type(mnist_train_dataset[0]))
ex_train_image, ex_train_label=mnist_train_dataset[0]
                                                   
print('Image shape',ex_train_image.shape,'label',ex_train_label,type(ex_train_label))

We can visualise the data using matplotlibs `imshow` function

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(ex_train_image[0,:,:])

Thus, we see that mnist_train_dataset contains 60k 28x28 images of hand-written numbers, with labels (the integer numbers from 0-9), in torch tensor format.

To use this data for deep learning, we can then load both test and train sets onto dataloaders, which dispatch batches and shuffle the data for us as:


In [ ]:
train_loader = torch.utils.data.DataLoader(
       mnist_train_dataset, batch_size= 128, shuffle = True)

test_loader = torch.utils.data.DataLoader(
       mnist_test_dataset, batch_size = 128, shuffle = False)

Shuffling is not required for test data.

Now that we have our dataloaders, we can simply iterate through them with the inbuilt iter() function.
<br>
To view just one batch instead of the entire data, we use apply the next() function on the iterator:

In [ ]:
im_batch, lab_batch=next(iter(train_loader)) # view one batch

print(len(im_batch),im_batch[0].shape,lab_batch[0])
plt.imshow(im_batch[0,0,:,:])

This returns one batch -  a tuple of images and labels. By looking at the image batch we can see N (first column) is 128 - the size of the batch. Image dminesions are $1 \times 28 \times 28$ as before.

### Loading custom data

The process for loading a customised dataset is not so far from the process above, except that we need to define our own dataset class. 


This class only needs two defined functions. One that provides the `len()` of the dataset, the other called `getitem` that outputs one data pair given an index.

*How* this is done is up to you. The method we go over is only a suggestion. However you choose to do it, `getitem` must, given an index, output the data and its appropriate label, corresponding to that index. This is because the DataLoader class will call this function.

In this example, we will use an example of image segmentation from computer vision, with a dataset which contains photos of scences accompanied by their semantic segmentations. Thus the label is now an image of size equal to that of the data.


We will read the images and segmentation masks from the ```sample_dataset_tutorial``` folder downloaded with your tutorial. Since they are .png files, we will need to import the imageio module to allow us to load them from this format. We will also need to import os to access the folders and directories in the computer via python.

In [ ]:
import imageio
import os

In [ ]:
class OurCustomDataSet(torch.utils.data.Dataset): #we create a class that inherits the torch Dataset abstract class 
    
    def __init__(self, data_folder_location): # initialise the class based on the folder containing the data
        
        """
        
        
        First look at the folder containing the data. It is structured in two folders.
        There are images in an 'images' folder, and masks in a 'segmentations' folder. 
        We need to match these up together and correctly.
        
        
        
        First we define the root folder and extract the images and segmentations subfolders 
        containing the data (images) and the labels (segmentations).
        
        
        
        """
        
        self.data_folder_location = data_folder_location
        
        self.images_folder = data_folder_location + '/images'
        
        self.labels_folder = data_folder_location + '/segmentations'
        
        """
        
        Now we use os.listdir() to get a list of the files within the subfolders. 
        
        We sort both these lists to make sure that the image files' titles and 
        the segmentation files' titles match. 
        
        They are titled in such a way to allow this to work by simply sorting. 
        
        (Other data might require more a more complex matching process.
        e.g. you might construct a dictionary matching image title to mask title)
        
        """
        
        
        
        self.images_list_sorted = os.listdir(self.images_folder)
        self.images_list_sorted.sort()
        
        self.labels_list_sorted = os.listdir(self.labels_folder)
        self.labels_list_sorted.sort()

        
        return
    
    def __len__(self):
        # this returns the length of the dataset. It is usually simple to code
        
        return len(os.listdir(self.images_folder))
    
    def __getitem__(self, idx):
        
        """
        
        getitem should pull the idx'th image and its associated segmentation and output them together as a sample
        
        we begin by indexing the list of image and label //titles// 
        
        """
        
        image_name = self.images_list_sorted[idx] # this is the idx'th image
        
        label_name = self.labels_list_sorted[idx] # this is the idx'th segmentation  # THEY SHOULD MATCH
        
        """
        
        Now we have to load the png files given these titles. 
        
        We first find their exact locations and then load them using imageio
        
        """
        
        full_image_location = str(self.images_folder)+'/' + str(image_name)
        
        full_label_location = str(self.labels_folder) + '/' + str(label_name)
        
        image = imageio.imread(full_image_location)
        
        label = imageio.imread(full_label_location)
        
        """
        
        Before we output, they are currently in .png format. So we need to turn them into torch Tensors
        
        """
        
        image = transforms.ToTensor()(image)
        label = transforms.ToTensor()(label)
             
        sample = image, label
        
        return sample

Now to try out our custom dataset.  Edit the below path so that it matches where you have placed your data. 

**Note** if you're using Google Colab you will need to load the data to your google drive and edit the path accordingly.

In [ ]:
directory = '../Data/sample_dataset_tutorial/'  # we define the directory of the data

ds = OurCustomDataSet(directory) # then we create the dataset 

In [ ]:
ds.__len__() # check if the length is correct. We can see from the folders there are 18 items in total
             # we should get the same result

In [ ]:
# Now lets try to pull out a sample. Lets try the first one.

example_im,ex_label=ds.__getitem__(0)
print(type(example_im),example_im.shape,ex_label.shape)

It worked! It returned two torch tensors, one labeled 'image' the other labeled 'label'.


If you are wish to be extra sure that they match, we can plot them 

In [ ]:
im, lab = OurCustomDataSet(directory).__getitem__(14) # im , lab are our image and label

In [ ]:
transforms.ToPILImage()(im) # quickly converting back to PIL image to view it

In [ ]:
transforms.ToPILImage()(lab)

We are done! At this point we simply need to load our dataset into the dataloader as usual and the dataloader will provide batches for our deep learning algorithm.

In [ ]:
loaded_data = DataLoader(ds, batch_size = 2, shuffle = False)

im_batch, lab_batch = next(iter(loaded_data)) # get a batch
print(im_batch.shape) # we get [2,3,1500,200] which we expect since the 
                      # images are [3 x 1500 x 2500] and the batch size is 2

## Exercise 4: Create a  custom data Loader for medical data

In this exercise we will create a custom data loader to load medical imaging data, and medical segmentations or phenotypic label data.

The data that we will use can be found in folder `dHCP_brain_data` and conatins 3D brain volumes, tissue segmentations and a pickled dataframe (`dHCP_demographics.pkl`) containing phenotypes including the infants age at birth ('birth_ga'), age at scan ('scan_ga') and gender ('gender'). Individual images can be identified by their subject 'id' (starting 'CC00') and the `session` of the scan (e.g. '7201'). This is reflected in filenames starting (for example) `sub-CC00050XX01_ses-7201` with files:
1. sub-CC00050XX01_ses-7201_T2w_restore_brain.nii.gz a T2-weighted structural image of each neonates brain
2. sub-CC00050XX01_ses-7201_drawem_tissue_labels.nii.gz a semantic segmentation of tissues in the brain, including labels for cortical and subcortical grey matter, white matter and cerebral spinal fluid (CSF). For the full list of labels see https://github.com/MIRTK/DrawEM/blob/master/label_names/tissue_labels_LUT_ITKSNAP.txt

For context, the image files look as 

<img src="brain_volumes.png" alt="Drawing" style="width: 500px;"/>

And can be viewed individually using 3D nifti image viewers such as FSLeyes https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSLeyes


### 4.1 Create a data loader for brain images and their segmentations

In the first task, as for the computer vision example we will create a dataloader to load brain images and their segmentations.

First we need a module that will load medical image volumes (here niftii images .nii.gz). There are several options including `SimpleITK`. Here, we use `nibabel`.

**Note** you will need to edit the paths throughout so that they match where you have put your data.

In [ ]:
import nibabel

image_file=nibabel.load('../Data/dHCP_brain_data/sub-CC00050XX01_ses-7201_T2w_restore_brain.nii.gz')
segmentation_file=nibabel.load('../Data/dHCP_brain_data/sub-CC00050XX01_ses-7201_drawem_tissue_labels.nii.gz')

img=image_file.get_fdata().astype(np.float)
seg=segmentation_file.get_fdata().astype(np.int32)
print(img.shape,seg.shape)
print(type(img),type(seg))

As we can see, loading data in this way returns a numpy array. Thus converting to a torch tensor is straightforward as

In [ ]:
img_tensor = torch.from_numpy(img).to(torch.float)
seg_tensor=torch.from_numpy(seg).to(torch.float)
print(img_tensor.shape)

Using `imshow` to view one central slice of the image and it's segmentation

In [ ]:
print(img_tensor.shape)
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True,
                       figsize=(8, 4))
ax[0].imshow(img_tensor[:,:,120], cmap=plt.cm.gray)
ax[0].set_title('T2 image - axial slice')
ax[1].imshow(seg_tensor[:,:,120], cmap=plt.cm.Dark2)
ax[1].set_title('Tissue segmentation')

However, as we are dealing with 3D volumesn we need to reshape our images into the form, which they are expected by Pytorch `conv3D` $C\times D \times H \times W $, where $C$=channels (here 1), $D$=depth, $H$=height and $W$ = width (https://pytorch.org/docs/master/nn.html#torch.nn.Conv3d). 

Hence, we need to bring our re-order the spatial dimensions of our tensor, using Pytorch `permute`0, 1)

In [ ]:
img_tensor=img_tensor.permute(2,0,1)
print(img_tensor.shape)

And add a fourth dimension for channels. This can be done using the PyTorch `unsqueeze` function

In [ ]:
img_tensor=img_tensor.unsqueeze(0)
print(img_tensor.shape)

Next we need a way of returning the paths of the data from an index. We could put images and segmentations into different folders (as above) and sort but this could feasibly lead to difficult to track down bugs. Let us instead use the project dataframe instead, using pandas:

In [ ]:
import pandas as pd

meta=pd.read_pickle('../Data/dHCP_brain_data/dHCP_demographics.pkl')
print(meta)

Each image file can thus be defined through:

In [ ]:
meta_sample=meta.iloc[0]
img_path=os.path.join('../Data/dHCP_brain_data','sub-' + str(meta_sample['id']) +'_ses-' + str(meta_sample['session']) +'_T2w_restore_brain.nii.gz')

img_sample=nibabel.load(img_path)

print(img_path)

**To do** Putting this all together, use these functions to create your own data loader for pairs of brain imaging data and clinical segmentations in the following steps

1.  complete the class constructor - used to initialise class variables: folder (path to data folder), meta (the dataframe) and transform
2. Complete the overloaded __len(self)__ class, to return the number of data examples
In `__getitem__`:
3.  return the paths to the image and the segmentation file
4. load the images
5. Convert to tensors and reshape to expected dimensions

**hint** consider also looking to https://pytorch.org/tutorials/beginner/data_loading_tutorial.html for guidance

In [ ]:
# students create own data loader

#we create a class that inherits the torch Dataset abstract class 
class BrainSegmentationDataset(torch.utils.data.Dataset): 
    
    # initialise the class based on the folder containing the data and the project dataframe
    def __init__(self, folder='', meta='',transform=None): 
        
       # 2.1.1 initialise the paths to the data folder, the data frame and define the transform operations
        self.folder = folder
        self.meta = pd.read_pickle(meta)
        self.transform=transform
        
    
    def __len__(self):
        
         # 2.1.2 return the number of examples in the dataset
        return len(self.meta)
    
    def __getitem__(self, idx):
        
        
        # 2.1.3 return the paths to the image and the segmentation file 
        meta_sample=self.meta.iloc[idx] # this will return the idx-th row from the dataframe
        image_name = os.path.join(self.folder,'sub-' + str(meta_sample['id']) +'_ses-' + 
                                  str(meta_sample['session']) +'_T2w_restore_brain.nii.gz')  # this is the idx'th image   
        label_name = os.path.join(self.folder,'sub-' + str(meta_sample['id']) +'_ses-' 
                                  + str(meta_sample['session']) +'_T2w_restore_brain.nii.gz')  # this is the idx'th segmentation  # THEY SHOULD MATCH
        
        # 2.1.4 load images 
        image=nibabel.load(image_name)
        label=nibabel.load(image_name)
        
        # 2.1.5 Convert to tensors and reshape to expected dimensions
        img_tensor = torch.from_numpy(img).to(torch.float)
        seg_tensor=torch.from_numpy(seg).to(torch.float)
        
        img_tensor=img_tensor.permute(2,0,1)
        seg_tensor=seg_tensor.permute(2,0,1)
        
        img_tensor=img_tensor.unsqueeze(0)
        seg_tensor=seg_tensor.unsqueeze(0)
        
        # convert to  tuple and return
        sample = img_tensor, seg_tensor

        if self.transform:
            sample = self.transform(sample)
            
        return sample

**To Do now try out your custom data** through:

1. Instantiate an instance of the BrainSegmentationDataset class by calling the constructor
2. check that the length is correct
3. return index 0 from getitem, 
4. plot the 120th axial slice of the returned img and segmentation. Be sure your segmentation matches your image  **Hint** remember that we have permuted the order of our slices and added a dimension for channels
5. Create a class iterator using DataLoader and create a batch of size 3, print the shape of the output


In [ ]:

#2.2.1 Inistantiate an instance of the BrainSegmentationDataset class by calling the constructor
directory = '../data/dHCP_brain_data'  # we define the directory of the data
meta='../data/dHCP_brain_data/dHCP_demographics.pkl'
ds = BrainSegmentationDataset(directory,meta) # then we create the dataset 

#2.2.2 check that the length is correct
print(ds.__len__())

#2.2.3 return index 0 from getitem
example_im,ex_label=ds.__getitem__(0)
print(type(example_im),example_im.shape,ex_label.shape)

#2.2.4 plot the 120th axial slice 
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True,
                       figsize=(8, 4))
ax[0].imshow(example_im[0,120,:,:], cmap=plt.cm.gray)
ax[0].set_title('T2 image - axial slice')
ax[1].imshow(ex_label[0,120,:,:], cmap=plt.cm.Dark2)
ax[1].set_title('Tissue segmentation')

#2.2.5 Create a class iterator using DataLoader and create a batch of size 3
loaded_data = DataLoader(ds, batch_size = 3, shuffle = False)

im_batch, lab_batch = next(iter(loaded_data)) # get a batch
print('batch shape', im_batch.shape) 

(Optional) Finally, using this example, repeat the process but this time for a classification task. I.e. rather than returning a semantic tissue segmentation instead create a data loaded for which the label is the neonataes age at scan.

In [3]:
# students create own data loader

class BrainAgeRegressionDataset(torch.utils.data.Dataset): 
    
    def __init__(self, folder='', meta='',label='',transform=None): 
        
       # 2.1.1 initialise the paths to the data folder, the data frame and define the transform operations
        self.folder = folder
        self.meta = pd.read_pickle(meta)
        self.label=label
        self.transform=transform
        
    
    def __len__(self):
        
         # 2.1.2 return the number of examples in the dataset
        return len(self.meta)
    
    def __getitem__(self, idx):
        
        
        # load image,  Convert to tensors and reshape to expected dimensions
        meta_sample=self.meta.iloc[idx] # this will return the idx-th row from the dataframe
        image_name = os.path.join(self.folder,'sub-' + str(meta_sample['id']) +'_ses-' + 
                                  str(meta_sample['session']) +'_T2w_restore_brain.nii.gz')  # this is the idx'th image   
        
        image=nibabel.load(image_name)
        
        img_tensor = torch.from_numpy(img).to(torch.float)
        img_tensor=img_tensor.permute(2,0,1)        
        img_tensor=img_tensor.unsqueeze(0)
        
        # read in label and convert to pytorch 
        label = torch.from_numpy(np.asarray(meta_sample[self.label])).to(torch.float)

        if self.transform:
            img_tensor = self.transform(img_tensor)
        
        # convert to  tuple and return
        sample = img_tensor, label

 
        return sample

NameError: name 'torch' is not defined

Test:

In [ ]:
#2.2.1 Instantiate 
directory = '../data/dHCP_brain_data'  # we define the directory of the data
meta='../data/dHCP_brain_data/dHCP_demographics.pkl'
label='scan_ga'
ds = BrainAgeRegressionDataset(directory,meta,label) # then we create the dataset 

#2.2.2 check that the length is correct
print(ds.__len__())

#2.2.3 return index 0 from getitem
example_im,ex_label=ds.__getitem__(0)
print(type(example_im),example_im.shape,ex_label)

loaded_data = DataLoader(ds, batch_size = 3, shuffle = False)

im_batch, lab_batch = next(iter(loaded_data)) # get a batch
print('batch shape', im_batch.shape,lab_batch) 

## References

For more reading on this topic see the official PyTorch tutorials https://pytorch.org/tutorials